<!--
03/11
Archivos binarios. Pickles.
Apareo de archivos secuenciales.
CLASE DE LABORATORIO 
(Gonzalo)
-->

# Archivos

Por lo general, cuando se trabaja con un archivo se hacen tres operaciones seguidas:

1. Abrir el archivo
2. Procesar el archivo
3. Cerrar el archivo

Y hay que tener cuidado, porque si ocurre algún error con el archivo en algún punto de su procesamiento es necesario encargarse de cerrarlo, antes de que la excepción siga subiendo niveles.

## Trabajando con archivo de una forma segura

Para trabajar con los archivos de una forma más simple es que se agregó la sentencia **with** que define un contexto dentro del cual nos asegura que, ocurra una excepción o no, el archivo se cerrará al momento de salir de ese contexto:

In [ ]:
try:
    with open('ejemplo.txt') as fd:
        a = 2
except NameError:
    print 'Ocurrio un error'
    
print fd.closed

In [ ]:
with open('ejemplo.txt') as archivo:
    print '¿El archivo se encuentra cerrado?: {}'.format(archivo.closed)
    print
    for linea in archivo:
        longitud = len(linea[:-1])
        print '%2d: %s' % (longitud, linea[:-1])

print
print '¿El archivo se encuentra cerrado?: {}'.format(archivo.closed)

Si bien no cerramos explícitamente el archivo usando la función close, al salir del bloque de código que encierra el with el archivo se encontrará cerrado.

## Pickles

Los [*pickles*](https://docs.python.org/2/library/pickle.html#module-pickle) son una forma de guardar estructuras de datos complejas y recuperarlas fácilmente, sin necesidad de convertirlas a texto y luego parsearlas:

### Ejemplo 1: Guardar de a un elemento

Se puede usar los pickles como se hacía con los viejos archivos de Pascal, donde se guardaba un registro detrás del otro; pero con la diferencia de que en este caso no es necesario que todos los registros sean del mismo tipo:

In [ ]:
import pickle  # Importo la biblioteca necesaria

# Creo la variable archivo
with open('ejemplo.pkl', 'wb') as archivo:
    pkl = pickle.Pickler(archivo)  # Creo mi punto de acceso a los datos a partir del archivo

    lista1 = [1, 2, 3]
    lista2 = [4, 5]
    diccionario = {'campo1': 1, 'campo2': 'dos'}

    pkl.dump(lista1)         # Guardo la lista1 de [1, 2, 3]
    pkl.dump(None)           # Guardo el valor None
    pkl.dump(lista2)
    pkl.dump('Hola mundo')
    pkl.dump('')
    pkl.dump(diccionario)
    pkl.dump(1)

Para leer de un archivo pickle no puedo usar el método readline que usa la estructura for, por lo que no me queda otra que siempre intentar leer hasta que lance una excepción del tipo *EOFError*:

In [ ]:
import pickle
with open('ejemplo.pkl', 'rb') as archivo:
    print pickle.load(archivo)  # lista1
    print pickle.load(archivo)  # None
    print pickle.load(archivo)  # lista2
    print pickle.load(archivo)  # Hola mundo
    print pickle.load(archivo)  # ''
    print pickle.load(archivo)  # diccionario
    print pickle.load(archivo)  # 1
    print pickle.load(archivo)  # ''
    

In [ ]:
with open('ejemplo.pkl', 'rb') as archivo:
    seguir_leyendo = True
    while seguir_leyendo:
        try:
            data = pickle.load(archivo)  # Leo del archivo un elemento
            print data
        except EOFError:
            seguir_leyendo = False


### Ejemplo 2: Guardo una lista de elementos

Así como en el ejemplo anterior guardamos de a un elemento por vez, también podríamos guardar una lista completa que tenga todos los elementos en memoria. De ésta forma, los archivos podrían usarse para cargar los datos al comenzar el programa y guardarlos todos juntos antes de terminar. <br>
Suponiendo que estoy desarrollando un juego en que no van a haber muchos jugadores compitiendo entre si, podría tener una lista con los puntajes y hacer:

In [ ]:
lista = [  # Creo la lista que quiero guardar
    {'usuario': 'csanchez', 'puntaje': 5}, 
    {'usuario': 'pperez', 'puntaje': 3}, 
    {'usuario': 'jromero', 'puntaje': 1}, 
]

# Guardo la lista en el archiv
with open('ejemplo_2.pkl', 'wb') as archivo:
    pkl = pickle.Pickler(archivo)
    pkl.dump(lista)

Y si ahora quiero sumarle 3 puntos a un usuario en particular tendría que:

1. Leer todo el archivo en una lista
2. Buscar el usuario y actualizarle los puntos
3. Guardar toda la lista en el archivo


In [20]:
from pprint import pprint

# Leo del archivo
with open('ejemplo_2.pkl', 'rb') as archivo:
    lista_puntajes = pickle.load(archivo)


# Actualizo el puntaje
print 'La lista antes de hacer el cambio es:'
pprint(lista_puntajes)

pos =  0

lista_puntajes[pos]['puntaje'] += 3

print 'La lista una vez hecho el cambio es:'
pprint(lista_puntajes)

# Guardo la lista en el archiv
with open('ejemplo_2.pkl', 'wb') as archivo:
    pkl = pickle.Pickler(archivo)
    pkl.dump(lista_puntajes)

La lista antes de hacer el cambio es:
[{'puntaje': 5, 'usuario': 'csanchez'},
 {'puntaje': 3, 'usuario': 'pperez'},
 {'puntaje': 1, 'usuario': 'jromero'}]
La lista una vez hecho el cambio es:
[{'puntaje': 8, 'usuario': 'csanchez'},
 {'puntaje': 3, 'usuario': 'pperez'},
 {'puntaje': 1, 'usuario': 'jromero'}]


Si bien es muy práctica esta alternativa, tiene el gran inconveniente de no hacer un uso eficiente de la memoria. <br>
Si el archivo contiene millones de usuarios, los estaríamos levantando todos a memoria, con el gran costo que tiene eso (no sólo en espacio, sino también en tiempo) con el único objetivo de sumarle 3 puntos a un único usuario. Y una vez que actualizamos el puntaje de ese usuario, tendríamos que volver a guardar todo el archivo en el disco.

## Abstrayendonos del uso de los pickles

Si bien el uso de los pickles puede resultar muy útil, la forma de leer la información guardada en ellos no suele ser muy cómoda. Por lo que podríamos implementar en un archivo `utils.py` las siguientes dos funciones para abstraernos un poco de cómo se accede a los datos:

```Python
# encoding: utf8
import pickle

def guardar_en_archivo(archivo, contenido):
    """Guarda lo que le pasen como segundo parámetro en el archivo 
    que recibe como primer parámetro.
    El parámetro llamado archivo tiene que estar abieto en modo 
    binario y para escritura (wb)
    """
    pickler = pickle.Pickler(archivo)
    pickler.dump(contenido)


def leer_desde_archivo(archivo):
    """Lee del archivo archivo un registro y lo retorna junto con una
    variable booleana que indica si llegó al fin de archivo o no.
    El parámetro llamado archivo tiene que estar abieto en modo 
    binario y para lectura (rb).
    Si se intenta leer más allá del fin de archivo, data valdrá None
    y fin_de_archivo será True. En cualquier otro caso fin_de_archivo
    será False.
    """
    try:
        data = pickle.load(archivo)
        fin_de_archivo = False
    except EOFError:
        data = None
        fin_de_archivo = True
    return data, fin_de_archivo

```

En este caso, se podría considerar que la función `leer_desde_archivo` funciona similar a cómo lo hacen los archivos con un registro centinella. <br>
Por lo que podríamos usar:

In [21]:
import utils


curso = [
    {'nombre': 'Sanchez, Lucas', 'nota': 8, 'padron': 90431, 'grupo': 1},
    {'nombre': 'Gigliotti, Emanuel', 'nota': 2, 'padron': 92953, 'grupo': 1},
    {'nombre': 'Aimar, Pablo', 'nota': 10, 'padron': 92407, 'grupo': 1},
    {'nombre': 'Alario, Lucas', 'nota': 8, 'padron': 96556, 'grupo': 2},
    {'nombre': 'Funes Mori, Rodrigo', 'nota': 7, 'padron': 92143, 'grupo': 2},
    {'nombre': 'Funes Mori, Javier', 'nota': 9, 'padron': 92431, 'grupo': 2},
    {'nombre': 'Aimar, Lucas', 'nota': 4, 'padron': 98306, 'grupo': 3},
    {'nombre': 'Sanchez, Carlos', 'nota': 8, 'padron': 97972, 'grupo': 3},
    {'nombre': 'Gago, Fernando', 'nota': 3, 'padron': 93108, 'grupo': 4},
    {'nombre': 'Kranneviter, Matias', 'nota': 5, 'padron': 96739, 'grupo': 5},
]

print 'Creo el archivo vacío usando el modo "wb"'
print 'Si tenía algo, ya lo borre...'
with open('curso.pkl', 'wb') as archivo:
    for alumno in curso:
        print 'Guardando el alumno {} en el archivo'.format(alumno)
        utils.guardar_en_archivo(archivo, alumno)

print
print 'Abro el archivo en modo lectura...'
with open('curso.pkl', 'rb') as archivo:
    alumno, fin_de_archivo = utils.leer_desde_archivo(archivo)
    while not fin_de_archivo:
        print 'Leyendo el alumno {} en el archivo'.format(alumno)
        alumno, fin_de_archivo = utils.leer_desde_archivo(archivo)


Creo el archivo vacío usando el modo "wb"
Si tenía algo, ya lo borre...
Guardando el alumno {'nombre': 'Sanchez, Lucas', 'grupo': 1, 'nota': 8, 'padron': 90431} en el archivo
Guardando el alumno {'nombre': 'Gigliotti, Emanuel', 'grupo': 1, 'nota': 2, 'padron': 92953} en el archivo
Guardando el alumno {'nombre': 'Aimar, Pablo', 'grupo': 1, 'nota': 10, 'padron': 92407} en el archivo
Guardando el alumno {'nombre': 'Alario, Lucas', 'grupo': 2, 'nota': 8, 'padron': 96556} en el archivo
Guardando el alumno {'nombre': 'Funes Mori, Rodrigo', 'grupo': 2, 'nota': 7, 'padron': 92143} en el archivo
Guardando el alumno {'nombre': 'Funes Mori, Javier', 'grupo': 2, 'nota': 9, 'padron': 92431} en el archivo
Guardando el alumno {'nombre': 'Aimar, Lucas', 'grupo': 3, 'nota': 4, 'padron': 98306} en el archivo
Guardando el alumno {'nombre': 'Sanchez, Carlos', 'grupo': 3, 'nota': 8, 'padron': 97972} en el archivo
Guardando el alumno {'nombre': 'Gago, Fernando', 'grupo': 4, 'nota': 3, 'padron': 93108} en el

## Apareo de archivos

El apareo de archivos consiste en tener un archivo con toda la información centralizada (comúnmente llamado archivo *maestro*) y en cierto momento es actualizado a partir de un segundo archivo llamado *novedades* generando un tercero con toda la información consolidada. <br>
Los archivos maestro y novedades están ordenados por la misma clave, por lo que el nuevo archivo maestro también debe quedar ordenado.
Por ejemplo, si contamos con un archivo llamado *cuentas.pkl* que en casa posición tiene la información correspondiente a una cuenta bancaria:

* **nro_cuenta**: Número de cuenta
* **tituar**: Titular de la cuenta
* **saldo**: Saldo de la cuenta
* **tipo_cuenta**: Tipo de cuenta
* **moneda**: Moneda en la cual opera la cuenta

Y uno que tenga las novedades diarias llamado *movimientos.pkl* con la siguiente información:

* **tipo**: Tipo de movimiento, es un string de una letra que puede ser A (alta), B (baja), M (modificación)
* **nro_cuenta**
* Si es:
  * *alta*(se asume saldo 0):
    * **titular**
    * **tipo_cuenta**
    * **moneda**
  * *modificación*:
    * **tipo_movimiento**: Un string que será una de las siguientes opciones: "credito" (cuando ingresa plata a la cuenta) o "debito" (cuando extraen plata de la cuenta)
    * **monto**: Monto a acreditar o debitar de la cuenta
  * *baja*: no es necesario agregar más campos



In [23]:
import random
import utils


CUENTAS_MAESTRO = (1, 2, 3, 5, 8, 9, 13, 15, 21, 25, 32)
CUENTAS_NOVEDADES = (4, 11, 16, 19)
CUENTAS = CUENTAS_MAESTRO + CUENTAS_NOVEDADES

def crear_cuenta(n):
    """Crea una cuenta con datos aleatorios."""
    cuenta = {
        'nro_cuenta': n,
        'titular': 'cliente_{}'.format(n),
        'tipo_cuenta': random.choice(('debito', 'corriente')),
        'moneda': '$',
    }
    
    return cuenta


def crear_cuentas():
    """Crea las cuentas del archivo maestro."""
    cuentas = []
    for n in CUENTAS_MAESTRO:
        cuenta = crear_cuenta(n)
        cuenta['saldo'] = random.randint(2000, 15000)
        cuentas.append(cuenta)
    
    return cuentas


def crear_nuevas_cuentas():
    """Crea las nuevas cuentas del archivo novedades."""
    cuentas = []
    for n in CUENTAS_NOVEDADES:
        cuenta = crear_cuenta(n)
        cuenta['tipo'] = 'A'
        cuentas.append(cuenta)
    
    return cuentas


def crear_movimientos():
    movimientos = []
    # Tomo 9 cuentas que sufriran las modificaciones
    for nro_cuenta in random.sample(CUENTAS_MAESTRO, 5):
        mov = {
            'tipo': 'M',
            'nro_cuenta': nro_cuenta,
            'tipo_movimiento': random.choice(('debito', 'credito')),
            'monto': random.randint(200, 900)
        }
        
        movimientos.append(mov)
    
    return movimientos


def buscar(lista, nro_cuenta):
    for x in lista:
        if x['nro_cuenta'] == nro_cuenta:
            return True
    
    return False

def crear_bajas(movimientos):
    bajas = []
    # Dare de baja 4 cuentas
    for nro_cuenta in random.sample(CUENTAS_MAESTRO, 7):
        b = {
            'tipo': 'B',
            'nro_cuenta': nro_cuenta,
        }
        
        # Me aseguro que en la lista de novedades
        # no se repiten registros
        if not buscar(movimientos, nro_cuenta): 
            bajas.append(b)
    
    return bajas


def crear_archivo_maestro():
    with open('cuentas.pkl', 'wb') as archivo:
        for cuenta in crear_cuentas():
            print 'Creando la cuenta {}'.format(cuenta)
            utils.guardar_en_archivo(archivo, cuenta)


def crear_archivo_novedades():
    nuevas_cuentas = crear_nuevas_cuentas()
    movimientos = crear_movimientos()
    bajas = crear_bajas(movimientos+nuevas_cuentas)
    orden = {
        'A': 0,
        'M': 1,
        'B': 2
    }
    novedades = sorted(
        nuevas_cuentas + movimientos + bajas,
        key=lambda x: (x['nro_cuenta'], orden[x['tipo']])
    )
    with open('movimientos.pkl', 'wb') as archivo:
        for nov in novedades:
            print 'Guardando la novedad {}'.format(nov)
            utils.guardar_en_archivo(archivo, nov)


################### Apareo ###################

def apareo():
    with open('cuentas.pkl', 'rb') as maestro, \
        open('movimientos.pkl', 'rb') as novedades, \
        open('nuevo.pkl', 'wb') as nuevo:
            cuenta, eof_ctas = utils.leer_desde_archivo(maestro)
            nov, eof_novs = utils.leer_desde_archivo(novedades)
            while not eof_ctas and not eof_novs:
                print 'Procesando cuenta nro {} y novedad {} del tipo {}'.format(
                    cuenta['nro_cuenta'], nov['nro_cuenta'], nov['tipo']
                )
                if nov['nro_cuenta'] < cuenta['nro_cuenta'] and nov['tipo'] == 'A':
                    # Si es un alta, acomodo el registro y lo guardo
                    del nov['tipo']
                    nov['saldo'] = 0
                    utils.guardar_en_archivo(nuevo, nov)
                    nov, eof_novs = utils.leer_desde_archivo(novedades)
                    
                    # No puede ser una B o M porque habría un error
                elif nov['nro_cuenta'] == cuenta['nro_cuenta']:
                    if nov['tipo'] == 'M':
                        # Si es una modificación, actualizo la cuenta, 
                        # guardo y leo de los dos archivos
                        monto = nov['monto'] if nov['tipo_movimiento'] == 'credito' else -1*nov['monto']
                        cuenta['saldo'] += monto
                        utils.guardar_en_archivo(nuevo, cuenta)

                    cuenta, eof_ctas = utils.leer_desde_archivo(maestro)
                    nov, eof_novs = utils.leer_desde_archivo(novedades)
                    
                    # Si fuera una B, tendría que ignorarlos y leer de 
                    # los archivos igual.
                    
                    # No puede ser una A porque habría un error
                elif nov['nro_cuenta'] > cuenta['nro_cuenta']:
                    # Si la novedad tiene un número de cuenta mayor, 
                    # significa que para esa cuenta no hubo novedades
                    # por lo que la guardo tal cual esta sin modificar
                    # y leo la siguiente
                    utils.guardar_en_archivo(nuevo, cuenta)
                    cuenta, eof_ctas = utils.leer_desde_archivo(maestro)
            
            # Como salí del while, termine con al menos uno de los
            # dos archivos, por lo que ahora puedeo leer lo que
            # quedaba y guardarlo casi tal cual vienen
            while not eof_ctas:
                print 'Procesando cuenta nro {}'.format(cuenta['nro_cuenta'])
                utils.guardar_en_archivo(nuevo, cuenta)
                cuenta, eof_ctas = utils.leer_desde_archivo(maestro)
                    
            while not eof_novs:
                print 'Procesando la novedad {} del tipo {}'.format(
                    nov['nro_cuenta'], nov['tipo']
                )
                del nov['tipo']
                nov['saldo'] = 0
                utils.guardar_en_archivo(nuevo, nov)
                nov, eof_novs = utils.leer_desde_archivo(novedades)


def mostrar_archivo_nuevo():
    print 'El archivo nuevo tiene los registros:'
    with open('nuevo.pkl', 'rb') as nuevo:
        cuenta, eof_ctas = utils.leer_desde_archivo(nuevo)
        while not eof_ctas:
            print cuenta
            cuenta, eof_ctas = utils.leer_desde_archivo(nuevo)
            


crear_archivo_maestro()
crear_archivo_novedades()
apareo()
mostrar_archivo_nuevo()

Creando la cuenta {'nro_cuenta': 1, 'saldo': 10248, 'moneda': '$', 'tipo_cuenta': 'debito', 'titular': 'cliente_1'}
Creando la cuenta {'nro_cuenta': 2, 'saldo': 7487, 'moneda': '$', 'tipo_cuenta': 'corriente', 'titular': 'cliente_2'}
Creando la cuenta {'nro_cuenta': 3, 'saldo': 14761, 'moneda': '$', 'tipo_cuenta': 'debito', 'titular': 'cliente_3'}
Creando la cuenta {'nro_cuenta': 5, 'saldo': 5197, 'moneda': '$', 'tipo_cuenta': 'corriente', 'titular': 'cliente_5'}
Creando la cuenta {'nro_cuenta': 8, 'saldo': 14196, 'moneda': '$', 'tipo_cuenta': 'corriente', 'titular': 'cliente_8'}
Creando la cuenta {'nro_cuenta': 9, 'saldo': 4733, 'moneda': '$', 'tipo_cuenta': 'debito', 'titular': 'cliente_9'}
Creando la cuenta {'nro_cuenta': 13, 'saldo': 10354, 'moneda': '$', 'tipo_cuenta': 'debito', 'titular': 'cliente_13'}
Creando la cuenta {'nro_cuenta': 15, 'saldo': 6832, 'moneda': '$', 'tipo_cuenta': 'corriente', 'titular': 'cliente_15'}
Creando la cuenta {'nro_cuenta': 21, 'saldo': 3757, 'moneda'

<!--
## JSON

Otra forma de guardar datos estructurados es usar un módulo llamado [json](https://docs.python.org/2/tutorial/inputoutput.html#saving-structured-data-with-json) y para esto se usan las funciones [dump](https://docs.python.org/2/library/json.html#json.dump) y [load](https://docs.python.org/2/library/json.html#json.load). <br>
Como ventaja tenemos que 
-->

# Ejercicios

1. Hacer el apareo, pero asumiendo que pueden venir más de un movimiento por cuenta (puede ser un alta, varios debitos/creditos e incluso una baja). 
1. Hacer un merge de dos archivos ordenados, que no es más que mezclar dos archivos del mismo tipo (por ejemplo, dos archivos maestro de cuentas bancarias) y generar un tercero donde se encuentren todos los registros de los primeros dos. 
1. Suponiendo que existe un archivo llamado utils.py donde se encuentran las funciones:

```Python
def guardar_en_archivo(archivo, contenido):
    """Guarda lo que le pasen como segundo parámetro en el archivo que
    recibe como primer parámetro.
    archivo tiene que estar abieto en modo binario y para escritura (wb)
    """
    ...


def leer_desde_archivo(archivo):
    """Lee del archivo archivo un registro y lo retorna junto con una
    variable booleana que indica si llegó al fin de archivo o no.
    archivo tiene que estar abieto en modo binario y para lectura (rb)
    """
    ...
    return data, fin_de_archivo
```
Leer dos archivos (61_matematica.dat y 75_computacion.dat) que tendrán registros con los campos:
    * padron
    * nombre
    * apellido
    * nota
    * codigo_departamento
    * codigo_materia
y armar uno nuevo donde sólo figuren las notas de los alumnos aprobados ordenados por padrón.<br>
Ambos archivos están ordenados por padrón y se deben leer una única vez. Como los archivos pueden ser muy grandes, no se pueden guardar en memoria.<br>
Una vez procesados los dos archivos se tienen que informar, para cada materia, cuántos alumnos aprobaron y cuántos desaprobaron.